In [1]:
import sys
#sys.path.append(r'H:\My Drive\2022\GitHub\pyMELTScalc\MELTS')
sys.path.append(r'G:\My Drive\Berkeley_NEW\MELTS_python_Paula\melts_matlab_git_master\package')

from meltsdynamic import MELTSdynamic
from meltsengine import MELTSengine
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#from meltsplotter import MELTSplotter
import sys, os
import shutil
import tempfile
from collections import OrderedDict
from pathlib import Path
import deepdish as dd
import itertools
%matplotlib inline
import time
import random
import pyMELTScalc as M
import re
from matplotlib.cm import get_cmap
from cmcrameri import cm
import Thermobar as pt
import distinctipy
import PySulfSat as ss

In [2]:
print(M.__version__),print(ss.__version__)

0.1.3
0.0.15


(None, None)

In [3]:
file = pd.read_csv('/Users/ollie/Documents/Cambridge_2020/Earth_Sciences/MELTs/pyMELTScalc-master/Examples/CrystallisationTests/SchmidtVan_scss.csv')
#file = pd.read_csv('/Users/ollie/Documents/Cambridge_2020/Earth_Sciences/Paper_2/FC_simulations/Metrich_Tan2318.csv')
#file = pd.read_csv('/Users/ollie/Rhiannon_MELTS/StVincbulks.csv')
#file = pd.read_csv('/Users/ollie/Rhiannon_MELTS/gleeson_azorestest.csv')
header = file.columns.values.tolist()
file

,P,Temp,FMQ,CO2,SiO2,TiO2,Al2O3,FeO,Fe2O3,MgO,...,Na2O,K2O,MnO,P2O5,H2O,Cr2O5,NiO,Ni,Cu,FeOt
0,2000,1350,0,0.1,49.2,0.69,12.9,7.2689,2.6907,12.93,...,1.99,0.97,0.2,0.2,1,0,0,20,30,9.9596


In [4]:
bulk = file.iloc[0].values.tolist()

h2o = [0.1,1.0, 4.0, 6.0]
emptydf = pd.DataFrame()

for i in range(len(h2o)):
    bulk[15] = h2o[i]

    emptydf = emptydf.append(pd.DataFrame([bulk]), ignore_index=True)

emptydf.columns = header
emptydf = emptydf[['SiO2', 'TiO2', 'Al2O3','FeOt', 'MgO', 'CaO', 'Na2O', 'K2O','MnO', 'P2O5', 'H2O']]#'CO2',,]], 
emptydf['Fe3Fet']=  [0.15,0.18,0.21,0.25]
#emptydf['CO2'] = [0.05]

In [5]:
emptydf.columns = [str(col) + '_Liq' for col in emptydf.columns]


In [6]:
# Create unbuffered input
unbuff = emptydf.copy()
unbuff['buffer'] = 'unbuffered'
unbuff['dictindex'] = 'H2O_' + unbuff['H2O_Liq'].astype(str)  + '_' 'buff_' + unbuff['Fe3Fet_Liq'].astype(str) #+ '_' 'CO2_' + merged['CO2_Liq'].astype(str)
unbuff

,SiO2_Liq,TiO2_Liq,Al2O3_Liq,FeOt_Liq,MgO_Liq,CaO_Liq,Na2O_Liq,K2O_Liq,MnO_Liq,P2O5_Liq,H2O_Liq,Fe3Fet_Liq,buffer,dictindex
0,49.2,0.69,12.9,9.9596,12.93,11.21,1.99,0.97,0.2,0.2,0.1,0.15,unbuffered,H2O_0.1_buff_0.15
1,49.2,0.69,12.9,9.9596,12.93,11.21,1.99,0.97,0.2,0.2,1.0,0.18,unbuffered,H2O_1.0_buff_0.18
2,49.2,0.69,12.9,9.9596,12.93,11.21,1.99,0.97,0.2,0.2,4.0,0.21,unbuffered,H2O_4.0_buff_0.21
3,49.2,0.69,12.9,9.9596,12.93,11.21,1.99,0.97,0.2,0.2,6.0,0.25,unbuffered,H2O_6.0_buff_0.25


In [7]:
Comp = unbuff.copy()

In [8]:
# Run 1 - unbuferred input fe3/fet via regression of global datasets

#P_lin = ([2000.0])  #np.array([2000.0, 4000.0]) doesnt work
P_lin = ([2000.0,1000.0,500.0])
P_results_unbuff = {}
for i in range(len(P_lin)):
    P_bar = P_lin[i]
    print(P_bar)
    Results = M.multi_path(cores=4,Model = "MELTSv1.2.0", comp = Comp, Frac_solid = True, Frac_fluid = True, T_end_C = 700, dt_C = 5,
                      P_bar = P_bar, Fe3Fet_Liq = None, H2O_Liq = None, find_liquidus = True)

    P_results_unbuff['P = ' + str(P_lin[i]) + ' bars'] = Results

2000.0


Running MELTSv1.2.0 calculations 0 to 3.0 ... Complete (time taken = 8.25 seconds)

1000.0


Running MELTSv1.2.0 calculations 0 to 3.0 ... Complete (time taken = 7.81 seconds)

500.0


Running MELTSv1.2.0 calculations 0 to 3.0 ... Complete (time taken = 7.52 seconds)



In [9]:
empty = pd.DataFrame() 

for key in P_results_unbuff:
    # p_dict is a dictionary of a single pressure 
    p_dict = P_results_unbuff[key]
    final_dict = p_dict
    for key in final_dict:
        new = final_dict[key]['All']
        new['dictindex'] = key
        new[['pressure_bar', 'T_C']] = final_dict[key]['Conditions'][['P_bar','T_C']]
        empty = empty.append(new)
empty.dictindex.unique()

array(['index = 0', 'index = 2', 'index = 3', 'index = 1'], dtype=object)

In [10]:
# replace dict_index values with H2O_CO2_fo2 identifiers
unbuff['index'] = unbuff.index #caveats w this format, change 
newkeys = unbuff[['index','dictindex']]
newkeys

,index,dictindex
0,0,H2O_0.1_buff_0.15
1,1,H2O_1.0_buff_0.18
2,2,H2O_4.0_buff_0.21
3,3,H2O_6.0_buff_0.25


In [11]:
empty_index = empty.reset_index()

In [12]:
empty_index['newentry'] = empty_index['dictindex'].str.extract('(\d+)')

In [13]:
empty_index['newcolumn'] = 0
for i in range(len(empty_index)):
    empty_index['newentry'][i] = float(empty_index['newentry'][i])
    empty_index['newcolumn'][i] = newkeys['dictindex'][empty_index['newentry'][i]]
empty_index.shape

<ipython-input-13-69e151919c31>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_index['newentry'][i] = float(empty_index['newentry'][i])
<ipython-input-13-69e151919c31>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_index['newcolumn'][i] = newkeys['dictindex'][empty_index['newentry'][i]]
/Users/ollie/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self

(1180, 272)

In [14]:
empty_index['unique'] = 'P_'+ empty_index['P_bar'].astype(str) + '_' + empty_index['newcolumn'].astype(str)

empty_index.newcolumn.unique(), empty_index.unique.unique() #if run at 2 pressures there should be double the categories of newcolumn
empty = empty_index.copy()

In [189]:
empty.to_excel('/Users/ollie/Documents/Cambridge_2020/Earth_Sciences/Paper_2/FC_simulations/Schmidt_FC_pyMELTcalc_1.2.0_natural_April_fluidinc.xlsx')


AttributeError: 'list' object has no attribute 'to_excel'

In [254]:
empty_na = pd.read_excel('/Users/ollie/Documents/Cambridge_2020/Earth_Sciences/Paper_2/FC_simulations/Schmidt_FC_pyMELTcalc_1.2.0_natural_April_fluidinc.xlsx', index_col=0)


In [255]:
empty_na.shape

(1180, 273)

In [256]:
empty_index = pd.concat([empty_na], axis=0,ignore_index=True)#,empty_na_shall

In [257]:
keepcols = empty_index.columns[empty_index.columns.str.contains('_Liq|pressure_bar|T_C|newcolumn')]
keep_ol_liq = empty_index.columns[empty_index.columns.str.contains('_Liq|pressure_bar|T_C|newcolumn|_Ol')]

In [258]:
slicedat = empty_index[keepcols]
Fo_Mg_MELTS = empty_index[keep_ol_liq]

In [259]:
#Fo_Mg_MELTS.to_excel('Fo_Mg_MELTS.xlsx') #to be worked on in MELTS versus EPMA Olivine-melt equilibria.ipynb

In [260]:
mass_cols = empty_index.columns[empty_index.columns.str.contains('index|mass|unique')]
syst_mass = empty_index[mass_cols]
syst_mass.head()

,index,mass_Liq,mass_Ol,mass_Cpx,mass_Plag,mass_Sp,mass_whitlockite1,mass_Apa,dictindex,mass_fluid1,mass_Cpx2,mass_rutile1,mass_Kspar,mass_Rhm,mass_tridymite1,unique
0,0,100.000000,0.000000,0.0,0.0,0.0,0.0,0.0,index = 0,NaN,NaN,NaN,NaN,NaN,NaN,P_2000.0_H2O_0.1_buff_0.15
1,1,99.409411,0.590589,0.0,0.0,0.0,0.0,0.0,index = 0,NaN,NaN,NaN,NaN,NaN,NaN,P_2000.0_H2O_0.1_buff_0.15
2,2,98.827986,0.582905,0.0,0.0,0.0,0.0,0.0,index = 0,NaN,NaN,NaN,NaN,NaN,NaN,P_2000.0_H2O_0.1_buff_0.15
3,3,98.257667,0.571799,0.0,0.0,0.0,0.0,0.0,index = 0,NaN,NaN,NaN,NaN,NaN,NaN,P_2000.0_H2O_0.1_buff_0.15
4,4,97.698188,0.560960,0.0,0.0,0.0,0.0,0.0,index = 0,NaN,NaN,NaN,NaN,NaN,NaN,P_2000.0_H2O_0.1_buff_0.15


In [261]:
column_to_move = syst_mass.pop("mass_fluid1") #put fluid as the final phase mass column
syst_mass.insert(14, "mass_fluid1", column_to_move)
syst_mass.head()

,index,mass_Liq,mass_Ol,mass_Cpx,mass_Plag,mass_Sp,mass_whitlockite1,mass_Apa,dictindex,mass_Cpx2,mass_rutile1,mass_Kspar,mass_Rhm,mass_tridymite1,mass_fluid1,unique
0,0,100.000000,0.000000,0.0,0.0,0.0,0.0,0.0,index = 0,NaN,NaN,NaN,NaN,NaN,NaN,P_2000.0_H2O_0.1_buff_0.15
1,1,99.409411,0.590589,0.0,0.0,0.0,0.0,0.0,index = 0,NaN,NaN,NaN,NaN,NaN,NaN,P_2000.0_H2O_0.1_buff_0.15
2,2,98.827986,0.582905,0.0,0.0,0.0,0.0,0.0,index = 0,NaN,NaN,NaN,NaN,NaN,NaN,P_2000.0_H2O_0.1_buff_0.15
3,3,98.257667,0.571799,0.0,0.0,0.0,0.0,0.0,index = 0,NaN,NaN,NaN,NaN,NaN,NaN,P_2000.0_H2O_0.1_buff_0.15
4,4,97.698188,0.560960,0.0,0.0,0.0,0.0,0.0,index = 0,NaN,NaN,NaN,NaN,NaN,NaN,P_2000.0_H2O_0.1_buff_0.15


In [262]:
syst_mass.iloc[:,2:15] #should be starting with mass_ol to the end including unique

,mass_Ol,mass_Cpx,mass_Plag,mass_Sp,mass_whitlockite1,mass_Apa,dictindex,mass_Cpx2,mass_rutile1,mass_Kspar,mass_Rhm,mass_tridymite1,mass_fluid1
0,0.000000,0.000000,0.0,0.0,0.0,0.000000,index = 0,NaN,NaN,NaN,NaN,NaN,NaN
1,0.590589,0.000000,0.0,0.0,0.0,0.000000,index = 0,NaN,NaN,NaN,NaN,NaN,NaN
2,0.582905,0.000000,0.0,0.0,0.0,0.000000,index = 0,NaN,NaN,NaN,NaN,NaN,NaN
3,0.571799,0.000000,0.0,0.0,0.0,0.000000,index = 0,NaN,NaN,NaN,NaN,NaN,NaN
4,0.560960,0.000000,0.0,0.0,0.0,0.000000,index = 0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,0.000000,0.035404,0.0,0.0,NaN,0.009264,index = 3,NaN,0.001911,0.220014,NaN,0.000000,0.006166
1176,0.000000,0.034547,0.0,0.0,NaN,0.008806,index = 3,NaN,0.001814,0.203996,NaN,0.000000,0.005730
1177,0.000000,0.034733,0.0,0.0,NaN,0.008389,index = 3,NaN,0.001725,0.190116,NaN,0.000000,0.005387
1178,0.000000,0.040921,0.0,0.0,NaN,0.009620,index = 3,NaN,0.001716,0.207352,NaN,0.033088,0.006982


In [263]:
# the mass crystallising at each step. 
cumulate_phase_mass = syst_mass.iloc[:,2:15].sum(axis=1)
cumulate_phase_mass

0       0.000000
1       0.590589
2       0.582905
3       0.571799
4       0.560960
          ...   
1175    0.272760
1176    0.254893
1177    0.240350
1178    0.299680
1179    0.000000
Length: 1180, dtype: float64

In [264]:
syst_mass = syst_mass.fillna(0)

In [265]:
syst_mass.columns[2:16] # we want column range starting with first phase to the end e.g. mass_Ol

Index(['mass_Ol', 'mass_Cpx', 'mass_Plag', 'mass_Sp', 'mass_whitlockite1',
       'mass_Apa', 'dictindex', 'mass_Cpx2', 'mass_rutile1', 'mass_Kspar',
       'mass_Rhm', 'mass_tridymite1', 'mass_fluid1', 'unique'],
      dtype='object')

In [266]:
# Calculate cumulative masses of solid phases, exlcuding index column 
for i in syst_mass.columns[2:15]:
    syst_mass[i + '_cumulative'] = syst_mass.groupby('unique')[i].transform(pd.Series.cumsum)

syst_mass[syst_mass.columns[15:]].head()

,unique,mass_Ol_cumulative,mass_Cpx_cumulative,mass_Plag_cumulative,mass_Sp_cumulative,mass_whitlockite1_cumulative,mass_Apa_cumulative,dictindex_cumulative,mass_Cpx2_cumulative,mass_rutile1_cumulative,mass_Kspar_cumulative,mass_Rhm_cumulative,mass_tridymite1_cumulative,mass_fluid1_cumulative
0,P_2000.0_H2O_0.1_buff_0.15,0.000000,0.0,0.0,0.0,0.0,0.0,index = 0,0.0,0.0,0.0,0.0,0.0,0.0
1,P_2000.0_H2O_0.1_buff_0.15,0.590589,0.0,0.0,0.0,0.0,0.0,index = 0index = 0,0.0,0.0,0.0,0.0,0.0,0.0
2,P_2000.0_H2O_0.1_buff_0.15,1.173494,0.0,0.0,0.0,0.0,0.0,index = 0index = 0index = 0,0.0,0.0,0.0,0.0,0.0,0.0
3,P_2000.0_H2O_0.1_buff_0.15,1.745292,0.0,0.0,0.0,0.0,0.0,index = 0index = 0index = 0index = 0,0.0,0.0,0.0,0.0,0.0,0.0
4,P_2000.0_H2O_0.1_buff_0.15,2.306252,0.0,0.0,0.0,0.0,0.0,index = 0index = 0index = 0index = 0index = 0,0.0,0.0,0.0,0.0,0.0,0.0


In [267]:
syst_mass[syst_mass.columns[16:29]].head() #select just the columns with mass_'phase'_cumulative, inc Liq or unique

,mass_Ol_cumulative,mass_Cpx_cumulative,mass_Plag_cumulative,mass_Sp_cumulative,mass_whitlockite1_cumulative,mass_Apa_cumulative,dictindex_cumulative,mass_Cpx2_cumulative,mass_rutile1_cumulative,mass_Kspar_cumulative,mass_Rhm_cumulative,mass_tridymite1_cumulative,mass_fluid1_cumulative
0,0.000000,0.0,0.0,0.0,0.0,0.0,index = 0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.590589,0.0,0.0,0.0,0.0,0.0,index = 0index = 0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.173494,0.0,0.0,0.0,0.0,0.0,index = 0index = 0index = 0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.745292,0.0,0.0,0.0,0.0,0.0,index = 0index = 0index = 0index = 0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.306252,0.0,0.0,0.0,0.0,0.0,index = 0index = 0index = 0index = 0index = 0,0.0,0.0,0.0,0.0,0.0,0.0


In [268]:
# mass of all solid phases + fluid 
syst_mass['mass_allphase_cumulative'] = syst_mass[syst_mass.columns[16:29]].sum(axis=1)
# mass of all solid phases + fluid + liq
syst_mass['mass_syst_unormtot'] = syst_mass['mass_allphase_cumulative'] + syst_mass['mass_Liq']

In [269]:
#### Alternative Calc#### calc both here but will only use one in model
# mass of all solid phases +NO fluid 
syst_mass['mass_solidphase_cumulative'] = syst_mass[syst_mass.columns[15:28]].sum(axis=1)
# mass of all solid phases + liq
syst_mass['mass_nofluid_syst_unormtot'] = syst_mass['mass_solidphase_cumulative'] + syst_mass['mass_Liq']

In [270]:
syst_mass[['mass_allphase_cumulative','mass_syst_unormtot', 'mass_solidphase_cumulative','mass_nofluid_syst_unormtot']]

,mass_allphase_cumulative,mass_syst_unormtot,mass_solidphase_cumulative,mass_nofluid_syst_unormtot
0,0.000000,100.000000,0.000000,100.000000
1,0.590589,100.000000,0.590589,100.000000
2,1.173494,100.001479,1.173494,100.001479
3,1.745292,100.002959,1.745292,100.002959
4,2.306252,100.004441,2.306252,100.004441
...,...,...,...,...
1175,89.630184,100.798758,84.334344,95.502918
1176,89.885077,100.809957,84.583507,95.508387
1177,90.125427,100.821162,84.818470,95.514206
1178,90.425107,100.832374,85.111168,95.518435


In [271]:
syst_mass[syst_mass.columns[15:]].head()

,unique,mass_Ol_cumulative,mass_Cpx_cumulative,mass_Plag_cumulative,mass_Sp_cumulative,mass_whitlockite1_cumulative,mass_Apa_cumulative,dictindex_cumulative,mass_Cpx2_cumulative,mass_rutile1_cumulative,mass_Kspar_cumulative,mass_Rhm_cumulative,mass_tridymite1_cumulative,mass_fluid1_cumulative,mass_allphase_cumulative,mass_syst_unormtot,mass_solidphase_cumulative,mass_nofluid_syst_unormtot
0,P_2000.0_H2O_0.1_buff_0.15,0.000000,0.0,0.0,0.0,0.0,0.0,index = 0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,100.000000,0.000000,100.000000
1,P_2000.0_H2O_0.1_buff_0.15,0.590589,0.0,0.0,0.0,0.0,0.0,index = 0index = 0,0.0,0.0,0.0,0.0,0.0,0.0,0.590589,100.000000,0.590589,100.000000
2,P_2000.0_H2O_0.1_buff_0.15,1.173494,0.0,0.0,0.0,0.0,0.0,index = 0index = 0index = 0,0.0,0.0,0.0,0.0,0.0,0.0,1.173494,100.001479,1.173494,100.001479
3,P_2000.0_H2O_0.1_buff_0.15,1.745292,0.0,0.0,0.0,0.0,0.0,index = 0index = 0index = 0index = 0,0.0,0.0,0.0,0.0,0.0,0.0,1.745292,100.002959,1.745292,100.002959
4,P_2000.0_H2O_0.1_buff_0.15,2.306252,0.0,0.0,0.0,0.0,0.0,index = 0index = 0index = 0index = 0index = 0,0.0,0.0,0.0,0.0,0.0,0.0,2.306252,100.004441,2.306252,100.004441


In [272]:
# move column mass_Liq next to the cumualtive columns - position may change based on number of phases 
col = syst_mass.pop('mass_Liq')
syst_mass.insert(15, 'mass_Liq', col)

In [273]:
syst_mass = syst_mass.drop([ 'dictindex_cumulative'], axis=1) #'unique_cumulative',


In [274]:
syst_mass.columns[15:]
syst_mass[syst_mass.columns[15:28]].tail() #from mass Liq to mass Fluid cumu

,mass_Liq,mass_Ol_cumulative,mass_Cpx_cumulative,mass_Plag_cumulative,mass_Sp_cumulative,mass_whitlockite1_cumulative,mass_Apa_cumulative,mass_Cpx2_cumulative,mass_rutile1_cumulative,mass_Kspar_cumulative,mass_Rhm_cumulative,mass_tridymite1_cumulative,mass_fluid1_cumulative
1175,11.168573,20.390059,28.127961,29.013771,3.813505,0.0,0.416929,0.0,0.074290,2.497829,0.0,0.000000,5.295840
1176,10.924879,20.390059,28.162508,29.013771,3.813505,0.0,0.425735,0.0,0.076104,2.701825,0.0,0.000000,5.301570
1177,10.695735,20.390059,28.197241,29.013771,3.813505,0.0,0.434124,0.0,0.077829,2.891940,0.0,0.000000,5.306957
1178,10.407267,20.390059,28.238162,29.013771,3.813505,0.0,0.443745,0.0,0.079545,3.099292,0.0,0.033088,5.313939
1179,0.000000,20.390059,28.238162,29.013771,3.813505,0.0,0.443745,0.0,0.079545,3.099292,0.0,0.033088,5.313939


In [275]:
# Normalise masses to total 100. to include fluid in this or not? 
# For now - yes March 2023. Discuss with Marie

#from liq_mass to final mass_'phase'_cumulative (fluid1)
for i in syst_mass.columns[15:28]:
    syst_mass[i + '_NORM'] = (syst_mass[i] * 100) / (syst_mass['mass_syst_unormtot'])
syst_mass.columns[28:]

Index(['mass_allphase_cumulative', 'mass_syst_unormtot',
       'mass_solidphase_cumulative', 'mass_nofluid_syst_unormtot',
       'mass_Liq_NORM', 'mass_Ol_cumulative_NORM', 'mass_Cpx_cumulative_NORM',
       'mass_Plag_cumulative_NORM', 'mass_Sp_cumulative_NORM',
       'mass_whitlockite1_cumulative_NORM', 'mass_Apa_cumulative_NORM',
       'mass_Cpx2_cumulative_NORM', 'mass_rutile1_cumulative_NORM',
       'mass_Kspar_cumulative_NORM', 'mass_Rhm_cumulative_NORM',
       'mass_tridymite1_cumulative_NORM', 'mass_fluid1_cumulative_NORM'],
      dtype='object')

In [276]:
syst_mass[syst_mass.columns[32:]].tail()

,mass_Liq_NORM,mass_Ol_cumulative_NORM,mass_Cpx_cumulative_NORM,mass_Plag_cumulative_NORM,mass_Sp_cumulative_NORM,mass_whitlockite1_cumulative_NORM,mass_Apa_cumulative_NORM,mass_Cpx2_cumulative_NORM,mass_rutile1_cumulative_NORM,mass_Kspar_cumulative_NORM,mass_Rhm_cumulative_NORM,mass_tridymite1_cumulative_NORM,mass_fluid1_cumulative_NORM
1175,11.080070,20.228483,27.905067,28.783858,3.783286,0.0,0.413625,0.0,0.073701,2.478035,0.0,0.000000,5.253874
1176,10.837104,20.226236,27.936236,28.780660,3.782866,0.0,0.422314,0.0,0.075493,2.680117,0.0,0.000000,5.258974
1177,10.608621,20.223987,27.967582,28.777462,3.782445,0.0,0.430589,0.0,0.077195,2.868386,0.0,0.000000,5.263733
1178,10.321355,20.221739,28.005055,28.774262,3.782025,0.0,0.440082,0.0,0.078888,3.073707,0.0,0.032815,5.270072
1179,0.000000,22.549113,31.228232,32.085969,4.217308,0.0,0.490732,0.0,0.087968,3.427468,0.0,0.036592,5.876619


In [277]:
syst_mass[syst_mass.columns[32:]].sum(axis=1) #check all liq and phases norm sum to 100

0       100.0
1       100.0
2       100.0
3       100.0
4       100.0
        ...  
1175    100.0
1176    100.0
1177    100.0
1178    100.0
1179    100.0
Length: 1180, dtype: float64

In [278]:
syst_mass[syst_mass.columns[33:]].head()

,mass_Ol_cumulative_NORM,mass_Cpx_cumulative_NORM,mass_Plag_cumulative_NORM,mass_Sp_cumulative_NORM,mass_whitlockite1_cumulative_NORM,mass_Apa_cumulative_NORM,mass_Cpx2_cumulative_NORM,mass_rutile1_cumulative_NORM,mass_Kspar_cumulative_NORM,mass_Rhm_cumulative_NORM,mass_tridymite1_cumulative_NORM,mass_fluid1_cumulative_NORM
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.590589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.173476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.745241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.306150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [279]:
##### DECISION CELL #####
# written to exclude fluid in normalisation
# so olivine should be 1 even if fluid exoslving to with olivine
syst_mass['mass_allphase_NORMcumu'] = syst_mass[syst_mass.columns[33:]].sum(axis=1)



In [280]:
syst_mass[syst_mass.columns[33:46]].head() #range of cumulative_norm mass phases

,mass_Ol_cumulative_NORM,mass_Cpx_cumulative_NORM,mass_Plag_cumulative_NORM,mass_Sp_cumulative_NORM,mass_whitlockite1_cumulative_NORM,mass_Apa_cumulative_NORM,mass_Cpx2_cumulative_NORM,mass_rutile1_cumulative_NORM,mass_Kspar_cumulative_NORM,mass_Rhm_cumulative_NORM,mass_tridymite1_cumulative_NORM,mass_fluid1_cumulative_NORM,mass_allphase_NORMcumu
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.590589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.590589
2,1.173476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.173476
3,1.745241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.745241
4,2.306150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.306150


In [281]:
# SOLID only mass cumulative: will be used if we exclude fluid
syst_mass['mass_solidphase_NORMcumu'] = syst_mass[syst_mass.columns[33:44]].sum(axis=1)

In [282]:
syst_mass[syst_mass.columns[33:46]] #range of solid phases no fluid

,mass_Ol_cumulative_NORM,mass_Cpx_cumulative_NORM,mass_Plag_cumulative_NORM,mass_Sp_cumulative_NORM,mass_whitlockite1_cumulative_NORM,mass_Apa_cumulative_NORM,mass_Cpx2_cumulative_NORM,mass_rutile1_cumulative_NORM,mass_Kspar_cumulative_NORM,mass_Rhm_cumulative_NORM,mass_tridymite1_cumulative_NORM,mass_fluid1_cumulative_NORM,mass_allphase_NORMcumu
0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,0.590589,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.590589
2,1.173476,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,1.173476
3,1.745241,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,1.745241
4,2.306150,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,2.306150
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,20.228483,27.905067,28.783858,3.783286,0.0,0.413625,0.0,0.073701,2.478035,0.0,0.000000,5.253874,88.919930
1176,20.226236,27.936236,28.780660,3.782866,0.0,0.422314,0.0,0.075493,2.680117,0.0,0.000000,5.258974,89.162896
1177,20.223987,27.967582,28.777462,3.782445,0.0,0.430589,0.0,0.077195,2.868386,0.0,0.000000,5.263733,89.391379
1178,20.221739,28.005055,28.774262,3.782025,0.0,0.440082,0.0,0.078888,3.073707,0.0,0.032815,5.270072,89.678645


In [283]:
# where ppn 1 refers to prpn of phases (solid + fluid1) excluding sulfide
# need columns from first and last phase_cumulative_norm masses - test search using cell above

for i in syst_mass.columns[33:46]:
    syst_mass[i + '_ppn1'] = (syst_mass[i]) / (syst_mass['mass_allphase_NORMcumu'])


In [284]:
syst_mass[syst_mass.columns[41:]].head()

,mass_Kspar_cumulative_NORM,mass_Rhm_cumulative_NORM,mass_tridymite1_cumulative_NORM,mass_fluid1_cumulative_NORM,mass_allphase_NORMcumu,mass_solidphase_NORMcumu,mass_Ol_cumulative_NORM_ppn1,mass_Cpx_cumulative_NORM_ppn1,mass_Plag_cumulative_NORM_ppn1,mass_Sp_cumulative_NORM_ppn1,mass_whitlockite1_cumulative_NORM_ppn1,mass_Apa_cumulative_NORM_ppn1,mass_Cpx2_cumulative_NORM_ppn1,mass_rutile1_cumulative_NORM_ppn1,mass_Kspar_cumulative_NORM_ppn1,mass_Rhm_cumulative_NORM_ppn1,mass_tridymite1_cumulative_NORM_ppn1,mass_fluid1_cumulative_NORM_ppn1,mass_allphase_NORMcumu_ppn1
0,0.0,0.0,0.0,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,0.590589,0.590589,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.173476,1.173476,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,1.745241,1.745241,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,2.306150,2.306150,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [285]:
syst_mass.mass_fluid1_cumulative_NORM_ppn1.iloc[[190,200]] #matches Marie_Model_check

190    0.283561
200    0.135135
Name: mass_fluid1_cumulative_NORM_ppn1, dtype: float64

### Modelling Sulfur

In [286]:
ONeill_MeasSulf=ss.calculate_O2021_SCSS(df=empty_index, T_K=empty_index['T_C']+273.15, 
P_kbar=empty_index['pressure_bar']/1000,Fe_FeNiCu_Sulf=0.65,Fe3Fet_Liq=empty_index['Fe3Fet_Liq'])
ONeill_MeasSulf.head()

Using inputted Fe_FeNiCu_Sulf ratio for calculations.


,SCSS2_ppm,LnS,Ln_a_FeO,Ln_a_FeS,DeltaG,LnCS2_calc,index,T_C,P_bar,h,...,CO2_tridymite1,Fe3Fet_tridymite1,h_tridymite1,mass_tridymite1,v_tridymite1,rho_tridymite1,newentry,newcolumn,unique,Fe_FeNiCu_Sulf_calc
0,1535.600994,7.336677,-2.364847,-0.497504,6.858216,-1.388881,0,1327.800000,2000,-1.193948e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,0,H2O_0.1_buff_0.15,P_2000.0_H2O_0.1_buff_0.15,0.65
1,1517.720543,7.324965,-2.362459,-0.497448,6.887392,-1.427439,1,1322.817460,2000,-1.195006e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,0,H2O_0.1_buff_0.15,P_2000.0_H2O_0.1_buff_0.15,0.65
2,1499.908347,7.313159,-2.360148,-0.497384,6.916761,-1.466366,2,1317.834921,2000,-1.188616e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,0,H2O_0.1_buff_0.15,P_2000.0_H2O_0.1_buff_0.15,0.65
3,1482.190398,7.301276,-2.357916,-0.497314,6.946325,-1.505651,3,1312.852381,2000,-1.182315e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,0,H2O_0.1_buff_0.15,P_2000.0_H2O_0.1_buff_0.15,0.65
4,1464.567792,7.289315,-2.355764,-0.497238,6.976085,-1.545296,4,1307.869841,2000,-1.176149e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,0,H2O_0.1_buff_0.15,P_2000.0_H2O_0.1_buff_0.15,0.65


In [287]:
# Li and Zhang 2022 ** chosen model **
LiZhang22=ss.calculate_LiZhang2022_SCSS(df=empty_index, T_K=empty_index['T_C']+273.15, 
P_kbar=empty_index['pressure_bar']/1000,Fe_FeNiCu_Sulf=0.65,Fe3Fet_Liq=empty_index['Fe3Fet_Liq'], H2O_Liq=empty_index['H2O_Liq'])
LiZhang22.head()

Using inputted Fe_FeNiCu_Sulf ratio for calculations.
replacing Fe3Fet_Liq in the original dataframe with that input into the function


,SCSS_Tot,index,T_C,P_bar,h,s,v,dvdp,logfO2,SiO2_Liq,...,Al_cat,Fe_cat,Mn_cat,Mg_cat,Ca_cat,Na_cat,K_cat,P_cat,H_cat,Fe3_cat
0,1193.493952,0,1327.800000,2000,-1.193948e+06,280.458347,36.866371,-0.000184,-6.786149,48.947950,...,0.138240,0.064445,0.001540,0.175264,0.109211,0.035082,0.011252,0.00154,0.0,0.011353
1,1177.183116,1,1322.817460,2000,-1.195006e+06,279.796338,36.839937,-0.000174,-6.825411,48.997428,...,0.139155,0.064391,0.001538,0.172530,0.109908,0.035314,0.011326,0.00155,0.0,0.011428
2,1160.977360,2,1317.834921,2000,-1.188616e+06,277.640382,36.629273,-0.000174,-6.864994,49.046880,...,0.140068,0.064332,0.001535,0.169812,0.110602,0.035546,0.011400,0.00156,0.0,0.011503
3,1144.904322,3,1312.852381,2000,-1.182315e+06,275.512322,36.421566,-0.000173,-6.904941,49.096118,...,0.140974,0.064266,0.001532,0.167119,0.111291,0.035776,0.011474,0.00157,0.0,0.011577
4,1128.964298,4,1307.869841,2000,-1.176149e+06,273.420621,36.217875,-0.000173,-6.945255,49.145142,...,0.141873,0.064193,0.001529,0.164453,0.111975,0.036004,0.011547,0.00158,0.0,0.011651


### SCAS calc

In [288]:
# Calculating SCAS 
ZT22_SCAS=ss.calculate_ZT2022_SCAS(df=empty_index, T_K=empty_index['T_C']+273.15)
ZT22_SCAS.head()

,SCAS6_ppm,index,T_C,P_bar,h,s,v,dvdp,logfO2,SiO2_Liq,...,H2O_tridymite1,CO2_tridymite1,Fe3Fet_tridymite1,h_tridymite1,mass_tridymite1,v_tridymite1,rho_tridymite1,newentry,newcolumn,unique
0,14338.942883,0,1327.800000,2000,-1.193948e+06,280.458347,36.866371,-0.000184,-6.786149,48.947950,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,H2O_0.1_buff_0.15,P_2000.0_H2O_0.1_buff_0.15
1,13906.964588,1,1322.817460,2000,-1.195006e+06,279.796338,36.839937,-0.000174,-6.825411,48.997428,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,H2O_0.1_buff_0.15,P_2000.0_H2O_0.1_buff_0.15
2,13490.915532,2,1317.834921,2000,-1.188616e+06,277.640382,36.629273,-0.000174,-6.864994,49.046880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,H2O_0.1_buff_0.15,P_2000.0_H2O_0.1_buff_0.15
3,13090.628678,3,1312.852381,2000,-1.182315e+06,275.512322,36.421566,-0.000173,-6.904941,49.096118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,H2O_0.1_buff_0.15,P_2000.0_H2O_0.1_buff_0.15
4,12705.343793,4,1307.869841,2000,-1.176149e+06,273.420621,36.217875,-0.000173,-6.945255,49.145142,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,H2O_0.1_buff_0.15,P_2000.0_H2O_0.1_buff_0.15


### S6/St

on 2023/04/03 added updated calcualtions from PW. Now S6/St is correct

In [289]:
# O'Neill and Mav 22 PW - from logfo2
calcS_OM2022_Givenfo2=ss.calculate_OM2022_S6St(df=empty_index, 
                                               T_K=empty_index['T_C']+273.15,
                                               logfo2=empty_index['logfO2'])
calcS_OM2022_Givenfo2.head()

,S6St_Liq,LnCS2_calc,LnCS6_calc,LnKSO2S2,LnS6S2,deltaQFM_calc,index,T_C,P_bar,h,...,Ca_Liq_cat_frac,Al_Liq_cat_frac,Na_Liq_cat_frac,K_Liq_cat_frac,Ti_Liq_cat_frac,P_Liq_cat_frac,Mg_Number_Liq_NoFe3,Mg_Number_Liq_Fe3,Fe2Fet_Liq_calc,Fe2_Liq_cat_frac
0,0.041105,-1.391840,12.076666,-14.633203,-3.149664,-13.854575,0,1327.800000,2000,-1.193948e+06,...,0.109211,0.138240,0.035082,0.011252,0.004719,0.00154,0.698086,0.731149,0.999964,0.075795
1,0.041990,-1.430449,12.134054,-14.740257,-3.127419,-13.942695,1,1322.817460,2000,-1.195006e+06,...,0.109908,0.139155,0.035314,0.011326,0.004750,0.00155,0.694700,0.728210,0.999966,0.075817
2,0.042892,-1.469428,12.191825,-14.847989,-3.105223,-14.031442,2,1317.834921,2000,-1.188616e+06,...,0.110602,0.140068,0.035546,0.011400,0.004782,0.00156,0.691278,0.725240,0.999968,0.075832
3,0.043803,-1.508765,12.250004,-14.956404,-3.083257,-14.120863,3,1312.852381,2000,-1.182315e+06,...,0.111291,0.140974,0.035776,0.011474,0.004812,0.00157,0.687833,0.722250,0.999969,0.075841
4,0.044723,-1.548463,12.308594,-15.065508,-3.061518,-14.210962,4,1307.869841,2000,-1.176149e+06,...,0.111975,0.141873,0.036004,0.011547,0.004843,0.00158,0.684365,0.719239,0.999970,0.075842


In [290]:
# O'Neill and Mav 22 from Fe3 - error 
calcS_OM2022_GivenFe3=ss.calculate_OM2022_S6St(df=empty_index, 
                                               T_K=empty_index['T_C']+273.15,
                                               Fe3Fet_Liq=empty_index['Fe3Fet_Liq'])

calcS_OM2022_GivenFe3.head()

,S6St_Liq,LnCS2_calc,LnCS6_calc,LnKSO2S2,LnS6S2,deltaQFM_calc,index,T_C,P_bar,h,...,Ca_Liq_cat_frac,Al_Liq_cat_frac,Na_Liq_cat_frac,K_Liq_cat_frac,Ti_Liq_cat_frac,P_Liq_cat_frac,Mg_Number_Liq_NoFe3,Mg_Number_Liq_Fe3,logfo2_calc,Fe2_Liq_cat_frac
0,0.427347,-1.391840,11.959676,-14.633203,-0.292685,0.928066,0,1327.800000,2000,-1.193948e+06,...,0.109211,0.138240,0.035082,0.011252,0.004719,0.00154,0.698086,0.731149,-6.140360,0.064445
1,0.425359,-1.430449,12.015920,-14.740257,-0.300812,0.931315,1,1322.817460,2000,-1.195006e+06,...,0.109908,0.139155,0.035314,0.011326,0.004750,0.00155,0.694700,0.728210,-6.185969,0.064391
2,0.423490,-1.469428,12.072544,-14.847989,-0.308463,0.934665,2,1317.834921,2000,-1.188616e+06,...,0.110602,0.140068,0.035546,0.011400,0.004782,0.00156,0.691278,0.725240,-6.231783,0.064332
3,0.421729,-1.508765,12.129572,-14.956404,-0.315680,0.938104,3,1312.852381,2000,-1.182315e+06,...,0.111291,0.140974,0.035776,0.011474,0.004812,0.00157,0.687833,0.722250,-6.277818,0.064266
4,0.420077,-1.548463,12.187010,-15.065508,-0.322456,0.941633,4,1307.869841,2000,-1.176149e+06,...,0.111975,0.141873,0.036004,0.011547,0.004843,0.00158,0.684365,0.719239,-6.324074,0.064193


In [291]:
# Jugo 2010 - didnt need correcting because no mix of parameter. always Frost
import Thermobar
buffers=pt.convert_fo2_to_buffer(fo2=10**empty_index['logfO2'], T_K=empty_index['T_C']+273.15, 
                    P_kbar=empty_index['P_bar']/1000)
buffers.head()
#empty_index['deltaQFM_Frost1991'] = buffers['deltaQFM_Frost1991']

using low temp


,deltaNNO_Frost1991,deltaQFM_Frost1991,QFM_equation_Choice,T_K,P_kbar,fo2,Cut off T (K)
0,-0.631582,0.017381,High T,1600.950000,2.0,1.636254e-07,896.15
1,-0.622408,0.026630,High T,1595.967460,2.0,1.494820e-07,896.15
2,-0.613252,0.035861,High T,1590.984921,2.0,1.364602e-07,896.15
3,-0.604154,0.045035,High T,1586.002381,2.0,1.244683e-07,896.15
4,-0.595114,0.054152,High T,1581.019841,2.0,1.134343e-07,896.15


In [292]:
Jugos6=ss.calculate_S6St_Jugo2010_eq10(deltaQFM=buffers['deltaQFM_Frost1991'])


### S_$total$ calcs

In [293]:
# Li and Zhang SCSS and S total via Oneill and Mavrogenes PW
df_ST_OM2022=ss.calculate_S_Total_SCSS_SCAS(SCSS=LiZhang22['SCSS_Tot'],
                                                      T_K=empty_index['T_C']+273.15,
                                                      SCAS=ZT22_SCAS['SCAS6_ppm'],
                                                      S6St_Liq=calcS_OM2022_Givenfo2['S6St_Liq'])
df_ST_OM2022.head()

,Total_S_ppm,S2_Tot_ppm,S6_Tot_ppm,deltaQFM,S6St_Liq,SCSS_2_ppm,SCAS_6_ppm,SCSS_Tot,SCAS_Tot,S6 in SCSS_Tot,S2 in SCAS_Tot
0,1244.654887,1193.493952,51.160934,None,0.041105,1193.493952,14338.942883,1244.654887,348841.074239,51.160934,334502.131357
1,1228.779944,1177.183116,51.596828,None,0.041990,1177.183116,13906.964588,1228.779944,331194.760255,51.596828,317287.795667
2,1213.005998,1160.977360,52.028638,None,0.042892,1160.977360,13490.915532,1213.005998,314529.885750,52.028638,301038.970218
3,1197.352148,1144.904322,52.447827,None,0.043803,1144.904322,13090.628678,1197.352148,298851.132157,52.447827,285760.503479
4,1181.818542,1128.964298,52.854243,None,0.044723,1128.964298,12705.343793,1181.818542,284090.924436,52.854243,271385.580643


In [294]:
# Li and Zhang SCSS and S total using O'Neill values OLD
df_ST_OM2022_fromFe3=ss.calculate_S_Total_SCSS_SCAS(SCSS=LiZhang22['SCSS_Tot'],
                                                      T_K=empty_index['T_C']+273.15,
                                                      SCAS=ZT22_SCAS['SCAS6_ppm'],
                                                      S6St_Liq=calcS_OM2022_GivenFe3['S6St_Liq'])
df_ST_OM2022_fromFe3.head()

,Total_S_ppm,S2_Tot_ppm,S6_Tot_ppm,deltaQFM,S6St_Liq,SCSS_2_ppm,SCAS_6_ppm,SCSS_Tot,SCAS_Tot,S6 in SCSS_Tot,S2 in SCAS_Tot
0,2084.147309,1193.493952,890.653356,None,0.427347,1193.493952,14338.942883,2084.147309,33553.423453,890.653356,19214.480570
1,2048.554125,1177.183116,871.371009,None,0.425359,1177.183116,13906.964588,2048.554125,32694.649437,871.371009,18787.684849
2,2013.802549,1160.977360,852.825188,None,0.423490,1160.977360,13490.915532,2013.802549,31856.516965,852.825188,18365.601433
3,1979.874997,1144.904322,834.970675,None,0.421729,1144.904322,13090.628678,1979.874997,31040.381630,834.970675,17949.752952
4,1946.749862,1128.964298,817.785564,None,0.420077,1128.964298,12705.343793,1946.749862,30245.246885,817.785564,17539.903092


In [295]:
#Li and Zhang SCSS and Nash S_tot
LiZha_ST_Nash=ss.calculate_S_Total_SCSS_SCAS(Fe3Fet_Liq=empty_index['Fe3Fet_Liq'],
    SCSS=LiZhang22['SCSS_Tot'],
    T_K=empty_index['T_C']+273.15,
    SCAS=ZT22_SCAS['SCAS6_ppm'], model='Nash')
LiZha_ST_Nash.head()

,Total_S_ppm,S2_Tot_ppm,S6_Tot_ppm,deltaQFM,S6St_Liq,SCSS_2_ppm,SCAS_6_ppm,SCSS_Tot,SCAS_Tot,S6 in SCSS_Tot,S2 in SCAS_Tot
0,3851.030463,1193.493952,2657.536511,None,0.690085,1193.493952,14338.942883,3851.030463,20778.531404,2657.536511,6439.588521
1,3757.140131,1177.183116,2579.957015,None,0.686681,1177.183116,13906.964588,3757.140131,20252.436163,2579.957015,6345.471574
2,3665.337367,1160.977360,2504.360007,None,0.683255,1160.977360,13490.915532,3665.337367,19745.067274,2504.360007,6254.151743
3,3575.129724,1144.904322,2430.225403,None,0.679759,1144.904322,13090.628678,3575.129724,19257.759235,2430.225403,6167.130557
4,3486.530460,1128.964298,2357.566162,None,0.676193,1128.964298,12705.343793,3486.530460,18789.533401,2357.566162,6084.189608


In [296]:
# Using Jugo values PW
LiZha_ST_Jugo=ss.calculate_S_Total_SCSS_SCAS(deltaQFM=buffers['deltaQFM_Frost1991'],
            SCSS=LiZhang22['SCSS_Tot'], SCAS=ZT22_SCAS['SCAS6_ppm'], model='Jugo')

#empty_index['Jugo_St']= LiZha_ST_Jugo['Total_S_ppm']
LiZha_ST_Jugo.head()
# the same as before.

,Total_S_ppm,S2_Tot_ppm,S6_Tot_ppm,deltaQFM,S6St_Liq,SCSS_2_ppm,SCAS_6_ppm,SCSS_Tot,SCAS_Tot,S6 in SCSS_Tot,S2 in SCAS_Tot
0,1203.764224,1193.493952,10.270272,0.017381,0.008532,1193.493952,14338.942883,1203.764224,1.680648e+06,10.270272,1.666309e+06
1,1187.753808,1177.183116,10.570692,0.026630,0.008900,1177.183116,13906.964588,1187.753808,1.562627e+06,10.570692,1.548720e+06
2,1171.855299,1160.977360,10.877938,0.035861,0.009283,1160.977360,13490.915532,1171.855299,1.453345e+06,10.877938,1.439854e+06
3,1156.094559,1144.904322,11.190238,0.045035,0.009679,1144.904322,13090.628678,1156.094559,1.352429e+06,11.190238,1.339339e+06
4,1140.471873,1128.964298,11.507575,0.054152,0.010090,1128.964298,12705.343793,1140.471873,1.259178e+06,11.507575,1.246473e+06


In [297]:
syst_mass['DeltaQFM_OMbuff'] = calcS_OM2022_Givenfo2['deltaQFM_calc']
syst_mass['DeltaQFM_OM_fromFe3'] = calcS_OM2022_GivenFe3['deltaQFM_calc']
syst_mass['DeltaQFM_JUGO'] = buffers['deltaQFM_Frost1991']
syst_mass['DeltaQFM_frostbuff'] = buffers['deltaQFM_Frost1991']

syst_mass['S_total_lizOM'] = df_ST_OM2022['Total_S_ppm']
syst_mass['S_total_lizOM_fromFe3'] = df_ST_OM2022_fromFe3['Total_S_ppm']
syst_mass['S_total_lizJUGO'] = LiZha_ST_Jugo['Total_S_ppm']

syst_mass['S6St_lizOM'] = df_ST_OM2022['S6St_Liq']
syst_mass['S6St_lizOM_fromFe3'] = df_ST_OM2022_fromFe3['S6St_Liq']
syst_mass['S6St_lizJUGO'] = LiZha_ST_Jugo['S6St_Liq']

syst_mass['SCSS']=LiZhang22['SCSS_Tot']
syst_mass['SCAS']=ZT22_SCAS['SCAS6_ppm']

In [298]:
# calc melt fraction 
syst_mass['melt_f'] = (syst_mass.mass_Liq_NORM) / 100

# replace nan with 0
syst_mass = syst_mass.replace(np.nan, 0)

In [299]:
DS = np.empty(len(syst_mass), dtype=float)
#(ppm) arc basalts globally lie within 0.1-0.2 wt% Muth and Wallace 2022
S_initial = 1500 

In [300]:
# not yet got an fo2 sensitive DS fluid-melt

DS_4000 = {1:3, 0.9:3, 0.8:3, 0.7:5, 0.6:8, 0.5:30, 0.4:50, 0.3:100, 0.2:200, 0.1:300, 0.0:300}

DS_2000 = {1:2, 0.9:2, 0.8:2, 0.7:5, 0.6:8, 0.5:30, 0.4:50, 0.3:100, 0.2:200, 0.1:300, 0.0:300}

DS_1600 = {1:1.1, 0.9:1.1, 0.8:1.1, 0.7:5, 0.6:8, 0.5:30, 0.4:50, 0.3:100, 0.2:200, 0.1:300, 0.0:300}

DS_800 = {1:8, 0.9:8, 0.8:8, 0.7:8, 0.6:8, 0.5:30, 0.4:50, 0.3:100, 0.2:200, 0.1:300, 0.0:300}

DS_dict = {'2000':DS_2000,'1000':DS_1600, '500':DS_800}

In [301]:
###### define truncate function
import math
def truncate(number, decimals=1):
    """
    Returns a value truncated to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer.")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more.")
    elif decimals == 0:
        return math.trunc(number)

    factor = 10.0 ** decimals
    return math.trunc(number * factor) / factor

In [302]:
# create extra column for melt fraction truncated to 1 dp
syst_mass['meltf_1dp1'] = 0 
meltf_1dp = []

for i in range(len(syst_mass)):
    x = syst_mass['melt_f'][i]
    syst_mass['meltf_1dp1'] = truncate(x, decimals=1)
    meltf_1dp.append(syst_mass['meltf_1dp1'].loc[i])
syst_mass['meltf_1dp1'] = meltf_1dp

In [303]:
empty = []
for key, val in DS_dict.items():
    print(key,val)
    dfnew = syst_mass[syst_mass.unique.str.contains(key)]
    
    dfnew['Dsulfur'] = dfnew['meltf_1dp1'].map(val)
    
    print(dfnew[['unique','melt_f','meltf_1dp1', 'Dsulfur']])
    
    empty.append(dfnew['Dsulfur'])
    
    flat_list = [item for sublist in empty for item in sublist]
    
    syst_mass['DS_fm'] = pd.Series(flat_list)

2000 {1: 2, 0.9: 2, 0.8: 2, 0.7: 5, 0.6: 8, 0.5: 30, 0.4: 50, 0.3: 100, 0.2: 200, 0.1: 300, 0.0: 300}
                         unique    melt_f  meltf_1dp1  Dsulfur
0    P_2000.0_H2O_0.1_buff_0.15  1.000000         1.0        2
1    P_2000.0_H2O_0.1_buff_0.15  0.994094         0.9        2
2    P_2000.0_H2O_0.1_buff_0.15  0.988265         0.9        2
3    P_2000.0_H2O_0.1_buff_0.15  0.982548         0.9        2
4    P_2000.0_H2O_0.1_buff_0.15  0.976938         0.9        2
..                          ...       ...         ...      ...
405  P_2000.0_H2O_1.0_buff_0.18  0.111533         0.1      300
406  P_2000.0_H2O_1.0_buff_0.18  0.108196         0.1      300
407  P_2000.0_H2O_1.0_buff_0.18  0.105175         0.1      300
408  P_2000.0_H2O_1.0_buff_0.18  0.102418         0.1      300
409  P_2000.0_H2O_1.0_buff_0.18  0.099875         0.0      300

[410 rows x 4 columns]
1000 {1: 1.1, 0.9: 1.1, 0.8: 1.1, 0.7: 5, 0.6: 8, 0.5: 30, 0.4: 50, 0.3: 100, 0.2: 200, 0.1: 300, 0.0: 300}
          

<ipython-input-303-968d0480f146>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfnew['Dsulfur'] = dfnew['meltf_1dp1'].map(val)
<ipython-input-303-968d0480f146>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfnew['Dsulfur'] = dfnew['meltf_1dp1'].map(val)
<ipython-input-303-968d0480f146>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [443]:
# Calc Bulk fluid-melt partition coeffcient of S
syst_mass['bulk_DS'] = syst_mass['DS_fm'] * syst_mass['mass_fluid1_cumulative_NORM_ppn1'] 

syst_mass['bulk_S_syst'] = S_initial / syst_mass['melt_f']

# Calc Melt S ppm concentration after fluid extracted
syst_mass['melt_S_nosulf'] = S_initial/ ((syst_mass['bulk_DS']*(1-syst_mass['melt_f'])+ syst_mass['melt_f']))

# Make new column filled with 1000 so we know if calcs have worked or not - ammend this later as not tidy
syst_mass['melt_S_sulf'] = syst_mass['melt_S_nosulf']
syst_mass['melt_S_sulf']  

0       1500.000000
1       1508.911460
2       1517.811154
3       1526.643605
4       1535.408831
           ...     
1175      94.503524
1176      94.426958
1177      94.355732
1178      94.260005
1179      85.082938
Name: melt_S_sulf, Length: 1180, dtype: float64

In [444]:
# Calc Melt S ppm for sulfide and fluid extraction
# changed desired Stotal calc in this cell
to_fill = []
u_list = syst_mass.unique.unique()

for u in u_list:
    df = syst_mass[syst_mass.unique == u]
    df = df.reset_index(drop=True)
    
    for i in range(0, len(df)):
        if syst_mass['melt_S_sulf'].loc[i] < syst_mass['S_total_lizOM_old'].loc[i]:
            syst_mass['melt_S_sulf'].loc[i] = syst_mass['melt_S_nosulf'].loc[i]
            to_fill.append(df['melt_S_sulf'].loc[i])
        
        if syst_mass['melt_S_sulf'].loc[i] >= syst_mass['S_total_lizOM_old'].loc[i]:
            syst_mass['melt_S_sulf'].loc[i] = syst_mass['S_total_lizOM_old'].loc[i]
            to_fill.append(df['melt_S_sulf'].loc[i])

print(len(to_fill)) #length of column rows

syst_mass['melt_S_sulf'] = to_fill

KeyError: 'S_total_lizOM_old'

In [ ]:
syst_mass[['melt_S_nosulf','melt_S_sulf','S_total_lizOM' ,'S_total_lizJUGO','S_total_lizOM_fromFe3']]

In [ ]:
syst_mass['fluid_S'] = syst_mass['bulk_S_syst'] - syst_mass['melt_S_nosulf']

In [ ]:
# Calculating Solid S 
# make decision on S total model used

syst_mass['solid_S'] = 0 

for i in range(0, len(syst_mass)):
        if syst_mass['melt_S_nosulf'].loc[i] < syst_mass['S_total_lizOM_old'].loc[i]:
            syst_mass['solid_S'].loc[i] = 0
        
        if syst_mass['melt_S_nosulf'].loc[i] >= syst_mass['S_total_lizOM_old'].loc[i]:
            syst_mass['solid_S'].loc[i] = syst_mass['melt_S_nosulf'].loc[i] - syst_mass['S_total_lizOM'].loc[i]


In [ ]:
syst_mass[['bulk_S_syst','S_total_lizOM','melt_S_sulf','melt_S_nosulf','fluid_S', 'solid_S']]

In [ ]:
syst_mass[['unique','meltf_1dp1','S_total_lizOM','DS_fm','melt_S_sulf','fluid_S','solid_S']] 

In [ ]:
# Xsulf = proportion of sulfide
Xsulf = syst_mass.solid_S / (36 * 10000)
syst_mass['Xsulf'] = Xsulf

In [ ]:
syst_mass[syst_mass.columns[47:59]].tail() #columns containing mass_'phase_norm_ppn1 includ fluid1

In [ ]:
syst_mass.mass_allphase_NORMcumu_ppn1

In [ ]:
# Calcualte proportions including the sulfide and fluid!
for i in syst_mass.columns[47:59]:
    syst_mass[i + '_ppn2'] = (syst_mass[i] / (syst_mass['mass_allphase_NORMcumu_ppn1'] + syst_mass['Xsulf']))#solidphase replaced allphase

syst_mass['Xsulf_NORM_ppn2'] = syst_mass['Xsulf']/ (syst_mass['mass_allphase_NORMcumu_ppn1'] + syst_mass['Xsulf'])

In [ ]:
syst_mass[['mass_Ol_cumulative_NORM_ppn1','mass_Ol_cumulative_NORM_ppn1_ppn2', 'Xsulf','Xsulf_NORM_ppn2','mass_allphase_NORMcumu_ppn1']]




In [ ]:
syst_mass[syst_mass.columns[79:]]

In [ ]:
propn = syst_mass[syst_mass.columns[79:]]
propn.columns = propn.columns.str.replace('ppn1_', '')


# Drop all columns names with ppn1_ppn2 and replace with propn dataframe - need a quicker method but is ok for now
syst_mass.drop(syst_mass.iloc[:, 79:], inplace=True, axis=1)

In [ ]:
metal_model = pd.concat([syst_mass, propn], axis=1)
metal_model.columns[76:] 

In [ ]:
metal_model.columns[82:]

In [ ]:
# should all be 1 if calcs were correct
metal_model['allphase_NORM_ppn2'] = metal_model[metal_model.columns[82:]].sum(axis=1)

In [ ]:
metal_model.head()

### Modeling Chlorine

In [ ]:
# Iterables for this section of the model

DCl = np.empty(len(metal_model), dtype=float)

Cl_initial = 0.5 #ppm, can be list of floats or single input


In [ ]:
# creating DCl dictionaries, keys = melt fraction and values are the raw DCl fluid/melt values
# wrong because 1-0.9 ... 0.1-0 rounding issues. think agagin with categorising
#trucncating is fine for now
DCl_2000 = {1:5, 0.9:6, 0.8:8, 0.7:10, 0.6:13, 0.5:16, 0.4:23, 0.3:30, 0.2:40, 0.1:55, 0.0:55}

DCl_1600 = {1:3, 0.9:4, 0.8:5, 0.7:7, 0.6:9, 0.5:11, 0.4:14, 0.3:20, 0.2:25, 0.1:31,0.0:31}

DCl_800 = {1:2, 0.9:2.5, 0.8:3, 0.7:3.5, 0.6:4, 0.5:5, 0.4:6, 0.3:7, 0.2:8, 0.1:10, 0.0:10}

DCl_dict = {'2000':DCl_2000,'1000':DCl_1600, '500':DCl_800 }

In [ ]:
###### define truncate function
import math
def truncate(number, decimals=1):
    """
    Returns a value truncated to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer.")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more.")
    elif decimals == 0:
        return math.trunc(number)

    factor = 10.0 ** decimals
    return math.trunc(number * factor) / factor

In [ ]:
# create extra column for melt fraction truncated to 1 dp
metal_model['meltf_1dp'] = 0 
meltf_1dp = []

for i in range(len(metal_model)):
    x = metal_model['melt_f'][i]
    metal_model['meltf_1dp'] = truncate(x, decimals=1)
    meltf_1dp.append(metal_model['meltf_1dp'].loc[i])
metal_model['meltf_1dp'] = meltf_1dp

In [ ]:
empty = []
for key, val in DCl_dict.items():
    dfnew = metal_model[metal_model.unique.str.contains(key)]
    dfnew['DCl'] = dfnew['meltf_1dp'].map(val)
    empty.append(dfnew['DCl'])
# creates a nested list     
empty[0]

In [ ]:
# flatten the multi-dimensional list
from collections import Iterable
def flatten(lis):
     for item in lis:
         if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item):
                 yield x
         else:        
             yield item
empty_flat = list(flatten(empty))

In [ ]:
metal_model['DCl_fm'] = empty_flat

In [ ]:
# Calc Bulk fluid-melt partition coeffcient of Cl
metal_model['bulk_DCl'] = metal_model['DCl_fm'] * metal_model['mass_fluid1_cumulative_NORM_ppn1']

# Calculate conc in bulk system i.e. completely incompatible behaviour of Cl
metal_model['bulk_Cl_syst'] = Cl_initial / metal_model['melt_f']
metal_model[['mass_fluid1_cumulative_NORM_ppn1','bulk_DCl','bulk_Cl_syst']]

In [ ]:
# Calc Melt S ppm concentration (sulfide absent FC)
metal_model['melt_Cl_raw'] = 0
metal_model['melt_Cl'] = 0
metal_model['solid_Cl'] = 0
metal_model['fluid_Cl'] = 0
metal_model['mass_bal_check'] = 0
metal_model['fluid_sal'] = 0
metal_model['mCl'] = 0
DCl_xxm = 0

for i in range(len(metal_model)):
    metal_model['melt_Cl_raw'].loc[i] = metal_model['bulk_Cl_syst'][i]/ ((metal_model['bulk_DCl'][i]*(1-metal_model['melt_f'][i])+ metal_model['melt_f'][i]))
    metal_model['melt_Cl'].loc[i] = metal_model['melt_f'][i] * metal_model['melt_Cl_raw'][i]
    metal_model['solid_Cl'].loc[i] = (metal_model['melt_Cl_raw'][i] * DCl_xxm) *(1- metal_model['melt_f'][i])
    metal_model['fluid_Cl'].loc[i] = (metal_model['melt_Cl_raw'][i] * metal_model['bulk_DCl'][i]) *(1- metal_model['melt_f'][i])
    metal_model['fluid_sal'].loc[i] = (metal_model['fluid_Cl'][i]*(58.44/35.453))
    metal_model['mCl'].loc[i] = ((metal_model['fluid_Cl'][i]*10000)/35.453/1000)
    metal_model['mass_bal_check'].loc[i] = (metal_model['fluid_Cl'][i] + metal_model['solid_Cl'][i] +metal_model['melt_Cl'][i])/metal_model['bulk_Cl_syst'][i]
    

### Calculating DX sulfide-liquid / melt 
#### • Li and Audetat 2015 experimental parameterisations


In [ ]:
metal_model = pd.concat([slicedat, metal_model],axis=1)

In [ ]:
metal_list = ['logDMo_s/m', 'logDZn_s/m', 'logDPb_s/m', 'logDCu_s/m','logDAg_s/m','logDAu_s/m']

for i in metal_list:
    metal_model[i] = 1 
#need to norm the wt% initially? do at earlier step
metal_model['logDMo_s/m'] = 8.66+(-0.94*(10000/(metal_model['T_C']+273.15)))+(-0.27)+(-2.34*np.log10(metal_model['FeOt_Liq'])) 
                                                                                   
metal_model['logDZn_s/m'] = 3.07+(-0.27*(10000/(metal_model['T_C']+273.15)))+(0)+(-1.24*np.log10(metal_model['FeOt_Liq']))                                                                                  

metal_model['logDPb_s/m'] = 2.61+(-0.05*(10000/(metal_model['T_C']+273.15)))+(-0.05)+(-0.86*np.log10(metal_model['FeOt_Liq']))

metal_model['logDCu_s/m'] = 1.13+(0.39*(10000/(metal_model['T_C']+273.15)))+(0.02)+(-0.86*np.log10(metal_model['FeOt_Liq']))

metal_model['logDAg_s/m'] = 2.01+(0.16*(10000/(metal_model['T_C']+273.15)))+(0)+(-0.16*np.log10(metal_model['FeOt_Liq']))

metal_model['logDAu_s/m'] = 3.18+(0.21*(10000/(metal_model['T_C']+273.15)))+(0.08)+(-0.82*np.log10(metal_model['FeOt_Liq']))
metal_model.head()                                                                            

In [445]:
## This will change dep on number of minerals crystalising
Kd_sulfmelt = metal_model[metal_model.columns[130:136]].copy()

In [446]:
Kd_sulfmelt.columns

Index(['logDMo_s/m', 'logDZn_s/m', 'logDPb_s/m', 'logDCu_s/m', 'logDAg_s/m',
       'logDAu_s/m'],
      dtype='object')

In [447]:
# remove the prefixes
Kd_sulfmelt.columns = Kd_sulfmelt.columns.str.replace(r'log', '')
raw_Kd_sm = Kd_sulfmelt[['DMo_s/m','DZn_s/m','DPb_s/m','DCu_s/m','DAg_s/m','DAu_s/m']]

In [448]:
# append to metal_model
metal_model = pd.concat([metal_model, raw_Kd_sm], axis=1)
metal_model

,T_C,SiO2_Liq,TiO2_Liq,Al2O3_Liq,Cr2O3_Liq,Fe2O3_Liq,FeO_Liq,FeOt_Liq,MnO_Liq,MgO_Liq,...,melt_Se,solid_Se,fluid_Se,massbal_se,DMo_s/m,DZn_s/m,DPb_s/m,DCu_s/m,DAg_s/m,DAu_s/m
0,1327.800000,48.947950,0.686465,12.833914,0.0,1.650774,8.422291,9.917007,0.198975,12.863760,...,0.100000,0.000000e+00,0.000000,pass,0.186956,0.147989,1.390798,2.729166,2.849986,3.754689
1,1322.817460,48.997428,0.690543,12.910160,0.0,1.660582,8.409581,9.913177,0.198491,12.654510,...,0.100593,9.467862e-07,0.000000,fail,0.169018,0.142932,1.389967,2.736916,2.853133,3.758922
2,1317.834921,49.046880,0.694606,12.986113,0.0,1.670351,8.396086,9.908528,0.197989,12.446781,...,0.101183,4.114862e-06,0.000000,pass,0.151049,0.137887,1.389161,2.744744,2.856305,3.763210
3,1312.852381,49.096118,0.698638,13.061488,0.0,1.680046,8.381863,9.903084,0.197471,12.241348,...,0.101766,9.946287e-06,0.000000,fail,0.133046,0.132851,1.388379,2.752650,2.859502,3.767552
4,1307.869841,49.145142,0.702639,13.136286,0.0,1.689667,8.366919,9.896851,0.196939,12.038203,...,0.102342,1.886041e-05,0.000000,pass,0.115008,0.127825,1.387621,2.760635,2.862725,3.771949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,785.290991,73.235375,0.148683,7.014109,0.0,0.055513,1.376598,1.426862,0.372440,0.001554,...,0.401902,0.000000e+00,0.500619,pass,-0.852240,0.327645,1.954838,4.701896,3.496956,5.117457
1176,780.273874,73.517895,0.140491,6.818679,0.0,0.050059,1.325213,1.370539,0.380748,0.000508,...,0.405424,0.000000e+00,0.517331,pass,-0.853610,0.337184,1.967631,4.734487,3.506954,5.141249
1177,775.256757,73.797632,0.132689,6.630334,0.0,0.044633,1.267791,1.308205,0.388905,0.000085,...,0.408796,0.000000e+00,0.533833,fail,-0.849007,0.349986,1.982745,4.769589,3.517457,5.167365
1178,770.239640,74.057025,0.125052,6.439050,0.0,0.038507,1.196689,1.231555,0.399685,0.000003,...,0.413083,0.000000e+00,0.555782,fail,-0.830761,0.370117,2.003002,4.810027,3.528991,5.198499


### Modeling Cu

In [449]:
# Define iterables for this section of the model
Cu_initial = 100 # ppm cant be iterated in loop 

DCu_cpx = 0.4
DCu_ol = 0.113
DCu_plag = 0.115
DCu_sp = 0.19
#DCu_sulf = 1000 replaced uniform value for the Li and Aduetat 2015 value

In [450]:
# Calculating DCu fluid/melt from Tattitch and Blundy 2017
metal_model['mass_fluid1_cumulative_NORM']
metal_model['h2o_mass_fluid1'] = metal_model['mass_fluid1_cumulative_NORM'] - (metal_model['fluid_Cl']*metal_model['mass_fluid1_cumulative_NORM']/100)- (((metal_model['fluid_S']/10000)*metal_model['mass_fluid1_cumulative_NORM']/100))
metal_model['Cl_mass_fluid1'] = (metal_model['fluid_Cl']*metal_model['mass_fluid1_cumulative_NORM']/100)
metal_model['S_mass_fluid1'] = ((metal_model['fluid_S']/10000)*metal_model['mass_fluid1_cumulative_NORM']/100)

# Calc mol fractions of each speices in the fluid
metal_model['XH2O_fluid1'] = (metal_model['h2o_mass_fluid1']/18)/((metal_model['h2o_mass_fluid1']/18)+(metal_model['Cl_mass_fluid1']/58.44)+(metal_model['S_mass_fluid1']/34.065))
metal_model['XNaCl_fluid1'] = (metal_model['Cl_mass_fluid1']/58.44)/((metal_model['h2o_mass_fluid1']/18)+(metal_model['Cl_mass_fluid1']/58.44)+(metal_model['S_mass_fluid1']/34.065))
metal_model['XH2S_fluid1'] = (metal_model['S_mass_fluid1']/34.065)/((metal_model['h2o_mass_fluid1']/18)+(metal_model['Cl_mass_fluid1']/58.44)+(metal_model['S_mass_fluid1']/34.065))

In [451]:
# Calc raw DCu fluid-melt
metal_model['DCu_fm'] = 80000*((metal_model['XNaCl_fluid1']**2)*(metal_model['XH2O_fluid1']**14)*(1+(180*(metal_model['XH2S_fluid1']))))+(380*metal_model['XNaCl_fluid1'])+0.8

metal_model['bulk_DCu_fm'] = (metal_model['mass_fluid1_cumulative_NORM_ppn1'].values) * (metal_model['DCu_fm'].values)
metal_model['bulk_DCu_fm'] = metal_model['bulk_DCu_fm'].fillna(0)
#replace nan with 0


In [452]:
# To calc trace metal distribution across liq-solid-fluid system we have devised a bulk 
# phase-melt Kd. Treating solid, fluid and melt as 3 proportions of the reservoir. 
# ** Maths needs checking ••

# divide by 1 because represents total system (liq+solid+fluid)
# do we have to change the mass to consider xsulf? or does xsulf jsut affect the proportions across that given weight?

metal_model['X_fluid1_system'] = metal_model['mass_fluid1_cumulative_NORM']/(100)
metal_model['X_solid_system'] = metal_model['mass_solidphase_NORMcumu']/(100)

metal_model['X_fluid1_system'], metal_model['X_solid_system'],metal_model['mass_Liq_NORM'] #should be 100

(0       0.000000
 1       0.000000
 2       0.000000
 3       0.000000
 4       0.000000
           ...   
 1175    0.052539
 1176    0.052590
 1177    0.052637
 1178    0.052701
 1179    0.058766
 Name: X_fluid1_system, Length: 1180, dtype: float64,
 0       0.000000
 1       0.005906
 2       0.011735
 3       0.017452
 4       0.023062
           ...   
 1175    0.836661
 1176    0.839039
 1177    0.841276
 1178    0.844086
 1179    0.941234
 Name: X_solid_system, Length: 1180, dtype: float64,
 0       100.000000
 1        99.409411
 2        98.826524
 3        98.254759
 4        97.693850
            ...    
 1175     11.080070
 1176     10.837104
 1177     10.608621
 1178     10.321355
 1179      0.000000
 Name: mass_Liq_NORM, Length: 1180, dtype: float64)

In [453]:
# Calc raw DCu xx-melt
bulk_DCu_xxm = []
bulk_DCu_phasem = []
for i in range(0, len(metal_model)):
    x = metal_model.loc[i]
    y = Kd_sulfmelt.loc[i]
    DCu_xxm = (x['mass_Ol_cumulative_NORM_ppn2']*DCu_ol)+(x['mass_Cpx_cumulative_NORM_ppn2']*DCu_cpx)+(x['mass_Cpx2_cumulative_NORM_ppn2']*DCu_cpx)+ (x['mass_Plag_cumulative_NORM_ppn2']*DCu_plag)+(x['mass_Sp_cumulative_NORM_ppn2']*DCu_sp)+ (x['Xsulf_NORM_ppn2']*y['DCu_s/m'])
    bulk_DCu_xxm.append(DCu_xxm)
    # new calc
    DCu_phasem =(x['bulk_DCu_fm']*x['X_fluid1_system']) + (x['X_solid_system']*DCu_xxm)
    bulk_DCu_phasem.append(DCu_phasem)



In [454]:
metal_model['bulk_DCu_xxm'] = bulk_DCu_xxm
metal_model['bulk_DCu_phasem'] = bulk_DCu_phasem

In [455]:
metal_model[['bulk_DCu_phasem','bulk_DCu_xxm']] = metal_model[['bulk_DCu_phasem','bulk_DCu_xxm']].replace(np.nan, 0)

In [456]:
metal_model[['mass_Ol_cumulative_NORM_ppn2', 'mass_Cpx_cumulative_NORM_ppn2',
       'mass_Plag_cumulative_NORM_ppn2', 'mass_Cpx2_cumulative_NORM_ppn2',
       'mass_Sp_cumulative_NORM_ppn2','Xsulf_NORM_ppn2','bulk_DCu_xxm','bulk_DCu_fm','X_fluid1_system','X_solid_system','bulk_DCu_phasem']]

,mass_Ol_cumulative_NORM_ppn2,mass_Cpx_cumulative_NORM_ppn2,mass_Plag_cumulative_NORM_ppn2,mass_Cpx2_cumulative_NORM_ppn2,mass_Sp_cumulative_NORM_ppn2,Xsulf_NORM_ppn2,bulk_DCu_xxm,bulk_DCu_fm,X_fluid1_system,X_solid_system,bulk_DCu_phasem
0,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,2.729166,0.000000,0.000000,0.000000,0.000000
1,0.999222,0.000000,0.000000,0.0,0.000000,0.000778,0.115040,0.000000,0.000000,0.005906,0.000679
2,0.999154,0.000000,0.000000,0.0,0.000000,0.000846,0.115226,0.000000,0.000000,0.011735,0.001352
3,0.999086,0.000000,0.000000,0.0,0.000000,0.000914,0.115412,0.000000,0.000000,0.017452,0.002014
4,0.999019,0.000000,0.000000,0.0,0.000000,0.000981,0.115598,0.000000,0.000000,0.023062,0.002666
...,...,...,...,...,...,...,...,...,...,...,...
1175,0.227491,0.313822,0.323705,0.0,0.042547,0.000000,0.196546,1.489861,0.052539,0.836661,0.242717
1176,0.226846,0.313317,0.322787,0.0,0.042426,0.000000,0.196142,1.567391,0.052590,0.839039,0.246999
1177,0.226241,0.312867,0.321927,0.0,0.042313,0.000000,0.195773,1.645923,0.052637,0.841276,0.251336
1178,0.225491,0.312282,0.320860,0.0,0.042173,0.000000,0.195305,1.753414,0.052701,0.844086,0.257260


In [457]:
# Calculate conc in bulk system i.e. completely incompatible behaviour of Cu
metal_model['bulk_Cu_syst'] = Cu_initial / metal_model['melt_f']
metal_model['melt_Cu_raw'] = metal_model['bulk_Cu_syst']/ ((metal_model['bulk_DCu_phasem']*(1-metal_model['melt_f']))+ metal_model['melt_f'])
metal_model['melt_Cu'] = metal_model['melt_f'] * metal_model['melt_Cu_raw']
metal_model['solid_Cu'] = metal_model['melt_Cu_raw']* (metal_model['X_solid_system']*metal_model['bulk_DCu_xxm']) *(1- metal_model['melt_f'])
metal_model['fluid_Cu'] = metal_model['melt_Cu_raw']* (metal_model['X_fluid1_system']*metal_model['bulk_DCu_fm'])*(1- metal_model['melt_f'])
   

In [458]:
massbal_cu = []
for i in range(0, len(metal_model)):
    x = (metal_model['fluid_Cu'][i] + metal_model['solid_Cu'][i] + metal_model['melt_Cu'][i]) / (metal_model['bulk_Cu_syst'][i])
    print(x)
    if x == 1:
        result = 'pass'
    else:
        result = 'fail'    
    print(x, result)
    massbal_cu.append(result)

1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
0.9999999999999999
0.9999999999999999 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
0.9999999999999998
0.9999999999999998 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
0.9999999999999999
0.9999999999999999 fail
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 p

1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
0.9999999999999999
0.9999999999999999 fail
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0000000000000002
1.0000000000000002 fail
0.9999999999999999
0.9999999999999999 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
0.9999999999999998
0.9999999999999998 fail
0.9999999999999999
0.9999999999999999 fail
nan
nan fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail


1.0
1.0 pass
0.9999999999999998
0.9999999999999998 fail
1.0000000000000002
1.0000000000000002 fail
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
nan
nan fail
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9

In [459]:
metal_model['massbal_cu'] = massbal_cu

In [460]:
metal_model['fluid_Cu'].tail()

1175    192.323736
1176    206.384943
1177    220.713224
1178    240.441424
1179           NaN
Name: fluid_Cu, dtype: float64

### Modeling Pb

In [461]:
# Define iterables for this section of the model
Pb_initial = 10 # ppm cant be iterated in loop 

DPb_cpx = 0.0
DPb_ol = 0.0
DPb_plag = 0.4


metal_model['mass_fluid1_cumulative_NORM_ppn1']

0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
          ...   
1175    0.059085
1176    0.058982
1177    0.058884
1178    0.058766
1179    0.058766
Name: mass_fluid1_cumulative_NORM_ppn1, Length: 1180, dtype: float64

In [462]:
# Calc raw and bulk DPb fluid-melt
metal_model['DPb_fm'] = 6.18 * metal_model['mCl']
# metal_model['mass_fluid1_cumulative_NORM_ppn1'] = 1 in this model because we treat fluid as seperate system
# caution - calculates for all - even if no fluid present. 
metal_model['bulk_DPb_fm'] = (metal_model['mass_fluid1_cumulative_NORM_ppn1'].values) * (metal_model['DPb_fm'].values)

In [463]:
# Calc raw DPb xx-melt
bulk_DPb_xxm = []
bulk_DPb_phasem = []
for i in range(0, len(metal_model)):
    x = metal_model.loc[i]
    y = Kd_sulfmelt.loc[i]
    DPb_xxm = (x['mass_Ol_cumulative_NORM_ppn2']*DPb_ol)+(x['mass_Cpx_cumulative_NORM_ppn2']*DPb_cpx)+(x['mass_Plag_cumulative_NORM_ppn2']*DPb_plag)+(x['mass_Cpx2_cumulative_NORM_ppn2']*DPb_cpx)+ (x['Xsulf_NORM_ppn2']*y['DPb_s/m'])
    bulk_DPb_xxm.append(DPb_xxm)
    DPb_phasem = (x['bulk_DPb_fm']*x['X_fluid1_system']) + (x['X_solid_system']*DPb_xxm) 
    bulk_DPb_phasem.append(DPb_phasem)
metal_model['bulk_DPb_xxm'] = bulk_DPb_xxm
metal_model['bulk_DPb_phasem'] = bulk_DPb_phasem

In [464]:
# Calculate conc in bulk system i.e. completely incompatible behaviour of Pb
metal_model['bulk_Pb_syst'] = Pb_initial / metal_model['melt_f']
metal_model['melt_Pb_raw'] = metal_model['bulk_Pb_syst']/ ((metal_model['bulk_DPb_phasem']*(1-metal_model['melt_f']))+ metal_model['melt_f'])
metal_model['melt_Pb'] = metal_model['melt_f'] * metal_model['melt_Pb_raw']
metal_model['solid_Pb'] = metal_model['melt_Pb_raw']* (metal_model['X_solid_system']*metal_model['bulk_DPb_xxm']) *(1- metal_model['melt_f'])
metal_model['fluid_Pb'] = metal_model['melt_Pb_raw']* (metal_model['X_fluid1_system']*metal_model['bulk_DPb_fm'])*(1- metal_model['melt_f'])


In [465]:
massbal_pb = []
for i in range(0, len(metal_model)):
    x = (metal_model['fluid_Pb'][i] + metal_model['solid_Pb'][i] + metal_model['melt_Pb'][i]) / (metal_model['bulk_Pb_syst'][i])
    print(x)
    if x == 1:
        result = 'pass'
    else:
        result = 'fail' 
        
        
    print(x, result)
    massbal_pb.append(result)
metal_model['massbal_pb'] = massbal_pb

1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0000000000000002
1.0000000000000002 fail
0.9999999999999999
0.9999999999999999 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0000000000000002
1.0000000000000002 fail
1.0000000000000002
1.0000000000000002 fail
0.9999999999999999
0.9999999999999999 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999998
0.9999999999999998 fail
0.9999999999999998
0.9999999999999998 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.00

1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.000000

1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
nan
nan fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999998
0.9999999999999998 fail
1.0000000000000002
1.0000000000000002 fail
0.9999999999999998
0.9999999999999998 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 

## Modelling Ag

In [466]:
# Define iterables for this section of the model
Ag_initial = 0.01 # ppm cant be iterated in loop 
DAg_cpx = 0.0
DAg_ol = 0.0
DAg_plag = 0.0

In [467]:
# fluid - melt Kd of Ag
metal_model['DAg_fm'] = 2.7 + (4 * metal_model['mCl'])
# metal_model['mass_fluid1_cumulative_NORM_ppn1'] = 1 in this model because we treat fluid as seperate system
# caution - calculates for all - even if no fluid present. 
metal_model['bulk_DAg_fm'] = (metal_model['mass_fluid1_cumulative_NORM_ppn1'].values) * (metal_model['DAg_fm'].values)

In [468]:
# solid - melt and allphase - melt Kd of Ag
bulk_DAg_xxm = []
bulk_DAg_phasem = []
for i in range(0, len(metal_model)):
    x = metal_model.loc[i]
    y = Kd_sulfmelt.loc[i]
    DAg_xxm = (x['Xsulf_NORM_ppn2']*y['DAg_s/m'])
    bulk_DAg_xxm.append(DAg_xxm)
    DAg_phasem = (x['bulk_DAg_fm']*x['X_fluid1_system']) + (x['X_solid_system']*DAg_xxm) 
    bulk_DAg_phasem.append(DAg_phasem)
metal_model['bulk_DAg_xxm'] = bulk_DAg_xxm
metal_model['bulk_DAg_phasem'] = bulk_DAg_phasem

In [469]:
# Calculate Ag distribution in melt-solid-fluid system: 
# Calculate conc in bulk system i.e. completely incompatible behaviour of Pb
metal_model['bulk_Ag_syst'] = Ag_initial / metal_model['melt_f']
metal_model['melt_Ag_raw'] = metal_model['bulk_Ag_syst']/ ((metal_model['bulk_DAg_phasem']*(1-metal_model['melt_f']))+ metal_model['melt_f'])
metal_model['melt_Ag'] = metal_model['melt_f'] * metal_model['melt_Ag_raw']
metal_model['solid_Ag'] = metal_model['melt_Ag_raw']* (metal_model['X_solid_system']*metal_model['bulk_DAg_xxm']) *(1- metal_model['melt_f'])
metal_model['fluid_Ag'] = metal_model['melt_Ag_raw']* (metal_model['X_fluid1_system']*metal_model['bulk_DAg_fm'])*(1- metal_model['melt_f'])

massbal_ag = []
for i in range(0, len(metal_model)):
    x = (metal_model['fluid_Ag'][i] + metal_model['solid_Ag'][i] + metal_model['melt_Ag'][i]) / (metal_model['bulk_Ag_syst'][i])
    print(x)
    if x == 1:
        result = 'pass'
    else:
        result = 'fail' 
        
        
    print(x, result)
    massbal_ag.append(result)
metal_model['massbal_ag'] = massbal_ag

1.0
1.0 pass
0.9999999999999998
0.9999999999999998 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999998
0.9999999999999998 fail
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999998
0.9999999999999998 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999998
0.9999999999999998 fail

1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
nan
nan fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0000000000000002
1.0000000000000002 fail
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0

1.0
1.0 pass
0.9999999999999998
0.9999999999999998 fail
0.9999999999999998
0.9999999999999998 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
0.9999999999999998
0.9999999999999998 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999998
0.9999999999999998 fail
1.0000000000000002
1.0000000000000002 fail
0.9999999999999999
0.9999999999999999 fail
nan
nan fail
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999998
0.9999999999999998 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass


## Modelling Au

In [470]:
# Define iterables for this section of the model
Au_initial = 0.001 # ppm cant be iterated in loop 
DAu_cpx = 0.0
DAu_ol = 0.0
DAu_plag = 0.0

In [471]:
# fluid - melt Kd of Ag
# needs more research, still havent defined this. 
metal_model['DAu_fm'] = 2 * (2.7 + (4 * metal_model['mCl']))
# metal_model['mass_fluid1_cumulative_NORM_ppn1'] = 1 in this model because we treat fluid as seperate system
# caution - calculates for all - even if no fluid present. 
metal_model['bulk_DAu_fm'] = (metal_model['mass_fluid1_cumulative_NORM_ppn1'].values) * (metal_model['DAu_fm'].values)

In [472]:
# solid - melt and allphase - melt Kd of Ag
bulk_DAu_xxm = []
bulk_DAu_phasem = []
for i in range(0, len(metal_model)):
    x = metal_model.loc[i]
    y = Kd_sulfmelt.loc[i]
    DAu_xxm = (x['Xsulf_NORM_ppn2']*y['DAu_s/m'])
    bulk_DAu_xxm.append(DAu_xxm)
    DAu_phasem = (x['bulk_DAu_fm']*x['X_fluid1_system']) + (x['X_solid_system']*DAu_xxm) 
    bulk_DAu_phasem.append(DAu_phasem)
metal_model['bulk_DAu_xxm'] = bulk_DAu_xxm
metal_model['bulk_DAu_phasem'] = bulk_DAu_phasem

In [473]:
# Calculate conc in bulk system i.e. completely incompatible behaviour of Au
metal_model['bulk_Au_syst'] = Au_initial / metal_model['melt_f']
metal_model['melt_Au_raw'] = metal_model['bulk_Au_syst']/ ((metal_model['bulk_DAu_phasem']*(1-metal_model['melt_f']))+ metal_model['melt_f'])
metal_model['melt_Au'] = metal_model['melt_f'] * metal_model['melt_Au_raw']
metal_model['solid_Au'] = metal_model['melt_Au_raw']* (metal_model['X_solid_system']*metal_model['bulk_DAu_xxm']) *(1- metal_model['melt_f'])
metal_model['fluid_Au'] = metal_model['melt_Au_raw']* (metal_model['X_fluid1_system']*metal_model['bulk_DAu_fm'])*(1- metal_model['melt_f'])

massbal_au = []
for i in range(0, len(metal_model)):
    x = (metal_model['fluid_Au'][i] + metal_model['solid_Au'][i] + metal_model['melt_Au'][i]) / (metal_model['bulk_Au_syst'][i])
    print(x)
    if x == 1:
        result = 'pass'
    else:
        result = 'fail' 
        
        
    print(x, result)
    massbal_au.append(result)
metal_model['massbal_au'] = massbal_au  

1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999998
0.9999999999999998 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999998
0.9999999999999998 fail
0.9999999999999998
0.9999999999999998 fail
0.9999999999999998
0.9999999999999998 fail
0.9999999999999998
0.9999999999999998 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999998
0.9999999999999998 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
0.9999999999999998
0.9999999999999998 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
0.9999999999999998
0.9999999999999998 fail
0.9999999999999998
0.9999999999999998 fail
0.9999999999999998
0.9999999999999998 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0


0.9999999999999998
0.9999999999999998 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
0.9999999999999998
0.9999999999999998 fail
1.0000000000000002
1.0000000000000002 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999998
0.9999999999999998 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999998
0.9999999999999998 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
0.9999999999999999
0.9999999999999999 fail
1.0000000000000002
1.0000000000000002 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999998
0.9999999999999998 fail
1.0


0.9999999999999999
0.9999999999999999 fail
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
0.9999999999999998
0.9999999999999998 fail
1.0
1.0 pass
nan
nan fail
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999998
0.9999999999999998 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
0.9999999999999998
0.9999999999999998 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999998
0.9999999999999998 fail
0.99

# Modelling Se

In [474]:
# Define iterables for this section of the model
Se_initial = 0.1 # ppm cant be iterated in loop 
DSe_cpx = 0.0
DSe_ol = 0.0
DSe_plag = 0.0

DSe_sulfPat = 345
DSe_sulfPeach = 1770

In [475]:
# fluid - melt Kd of Ag
# needs more research, still havent defined this. 
metal_model['DSe_fm'] = 50
# metal_model['mass_fluid1_cumulative_NORM_ppn1'] = 1 in this model because we treat fluid as seperate system
# caution - calculates for all - even if no fluid present. 
metal_model['bulk_DSe_fm'] = (metal_model['mass_fluid1_cumulative_NORM_ppn1'].values) * (metal_model['DSe_fm'].values)

In [476]:
# solid - melt and allphase - melt Kd of Ag
bulk_DSe_xxm_Pat = []
bulk_DSe_xxm_Peach = []
bulk_DSe_phasem_Pat = []
bulk_DSe_phasem_Peach = []
for i in range(0, len(metal_model)):
    x = metal_model.loc[i]
    DSe_xxm_Pat = (x['Xsulf_NORM_ppn2']*DSe_sulfPat)
    DSe_xxm_Peach = (x['Xsulf_NORM_ppn2']*DSe_sulfPeach)
    bulk_DSe_xxm_Pat.append(DSe_xxm_Pat)
    bulk_DSe_xxm_Peach.append(DSe_xxm_Peach)
    
    DSe_phasem_Pat = (x['bulk_DSe_fm']*x['X_fluid1_system']) + (x['X_solid_system']*DSe_xxm_Pat) 
    DSe_phasem_Peach = (x['bulk_DSe_fm']*x['X_fluid1_system']) + (x['X_solid_system']*DSe_xxm_Peach)
    bulk_DSe_phasem_Pat.append(DSe_phasem_Pat)
    bulk_DSe_phasem_Peach.append(DSe_phasem_Peach)
    
# Patten Kds
metal_model['bulk_DSe_xxm'] = bulk_DSe_xxm_Pat
metal_model['bulk_DSe_phasem'] = bulk_DSe_phasem_Pat

# Peach Kds
metal_model['bulk_DSe_xxm_Peach'] = bulk_DSe_xxm_Peach
metal_model['bulk_DSe_phasem_Peach'] = bulk_DSe_phasem_Peach

# Calculate Ag distribution in melt-solid-fluid system: 
metal_model['bulk_Se_syst'] = Se_initial / metal_model['melt_f']

In [477]:
# Running the following for Paten value of 345
# Calculate conc in bulk system i.e. completely incompatible behaviour of Au
metal_model['bulk_Se_syst'] = Se_initial / metal_model['melt_f']
metal_model['melt_Se_raw'] = metal_model['bulk_Se_syst']/ ((metal_model['bulk_DSe_phasem']*(1-metal_model['melt_f']))+ metal_model['melt_f'])
metal_model['melt_Se'] = metal_model['melt_f'] * metal_model['melt_Se_raw']
metal_model['solid_Se'] = metal_model['melt_Se_raw']* (metal_model['X_solid_system']*metal_model['bulk_DSe_xxm']) *(1- metal_model['melt_f'])
metal_model['fluid_Se'] = metal_model['melt_Se_raw']* (metal_model['X_fluid1_system']*metal_model['bulk_DSe_fm'])*(1- metal_model['melt_f'])

massbal_se = []
for i in range(0, len(metal_model)):
    x = (metal_model['fluid_Se'][i] + metal_model['solid_Se'][i] + metal_model['melt_Se'][i]) / (metal_model['bulk_Se_syst'][i])
    print(x)
    if x == 1:
        result = 'pass'
    else:
        result = 'fail' 
        
        
    print(x, result)
    massbal_se.append(result)
metal_model['massbal_se'] = massbal_se  

1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0000000000000002
1.0000000000000002 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
0.9999999999999999
0.9999999999999999 fail
0.9999999999999997
0.9999999999999997 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 pass


1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
0.9999999999999999
0.9999999999999999 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0000000000000002
1.0000000000000002 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0000000000000002
1.0000000000000002 fail
0.9999999999999999
0.9999999999999999 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0000000000000002
1.0000000000000002 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
0.9999999999999998
0.9999999999999998 fail
1.0
1.0 pass
1.0000000000000002
1.00000000000000

1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
nan
nan fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 pass
1.0000000000000002
1.0000000000000002 fail
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
0.9999999999999999
0.9999999999999999 fail
0.9999999999999999
0.9999999999999999 fail
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.9999999999999999 fail
0.9999999999999999
0.9999999999999999 fail
1.0000000000000002
1.0000000000000002 fail
1.0000000000000002
1.0000000000000002 fail
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
1.0
1.0 pass
0.9999999999999999
0.999999999

In [478]:
#metal_model.to_excel('Apriledit_MetalModel_noCO2_incFluid_ONEILL_changed_Metali_concs.xlsx')

#metal_model.to_excel('Apriledit_MetalModel_noCO2_incFluid_JUGO_changed_Metali_concs.xlsx')

#metal_model.to_excel('Apriledit_MetalModel_noCO2_incFluid_ONEILL_OLD_changed_Metali_concs.xlsx')
